# LangChain Runnable 컴포넌트

LangChain에서는 다양한 Runnable 컴포넌트를 통해 체인을 구성하고 실행할 수 있습니다. 이 노트북에서는 다음 주요 컴포넌트들을 살펴보겠습니다:

- 일반적인 체인 구성
- RunnablePassthrough
- RunnableParallel
- 복합 체인 구성 예시

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 일반적인 체인 구성

LangChain에서는 파이프 연산자(`|`)를 사용하여 체인을 구성할 수 있습니다. 이는 각 컴포넌트의 출력이 다음 컴포넌트의 입력으로 전달되는 방식입니다.

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 제곱은?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm

In [4]:
# 체인에 입력값을 딕셔너리 형태로 전달합니다
chain.invoke({"num": 5})

AIMessage(content='5의 제곱은 25입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c687559f-8a3c-45b5-b53b-b8082587b4e7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26})

In [6]:
# 값이 하나일 때는 딕셔너리가 아니어도 작동하지만, 둘 이상일 때는 에러가 발생합니다
chain.invoke(5)

AIMessage(content='5의 제곱은 25입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1331c75-f487-4366-b97f-20bdd1520258-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26})

## RunnablePassthrough

`RunnablePassthrough`는 입력 데이터를 변경하지 않고 그대로 통과시키거나, 추가 키를 더하여 전달할 수 있는 컴포넌트입니다.

- `RunnablePassthrough()` 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)` 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

In [9]:
# RunnablePassthrough에 넘기면 그대로 딕셔너리로 반환됩니다
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [10]:
# 사용 예시: 숫자를 입력받아 딕셔너리 형태로 변환하여 체인에 전달
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 15, 'total_tokens': 19, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-70607103-5469-419f-91fd-dc07bd565978-0', usage_metadata={'input_tokens': 15, 'output_tokens': 4, 'total_tokens': 19})

In [13]:
# RunnablePassthrough.assign을 사용하면 새로운 키-값 쌍이 할당되어 합쳐집니다
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## RunnableParallel

`RunnableParallel`은 여러 Runnable 컴포넌트를 병렬로 실행할 수 있게 해주는 모듈입니다. 각 컴포넌트는 서로 독립적으로 실행되며, 결과는 지정된 키와 함께 딕셔너리 형태로 반환됩니다.

In [14]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## 복합 체인 구성 예시

여기서는 RunnablePassthrough와 RunnableParallel을 조합하여 복합적인 체인을 구성하는 예시를 살펴보겠습니다.

In [15]:
# 국가의 수도를 묻는 체인
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)

# 국가의 면적을 묻는 체인
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [16]:
# RunnableParallel을 사용하여 두 체인을 병렬로 실행
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 19, 'total_tokens': 25, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d1783999-5e5a-4fe2-8395-7197f50579dd-0', usage_metadata={'input_tokens': 19, 'output_tokens': 6, 'total_tokens': 25}),
 'area': AIMessage(content='대한민국의 면적은 총 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 20, 'total_tokens': 44, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_predi